A partir de una tabla de datos ya procesada para utilizar con los modelos teoricos, 
en este nb buscamos optimizar

In [1]:
## seteamos los parametros del notebook
%autosave 180

import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
#import matplotlib
#%matplotlib inline
import sys

np.random.seed(2015)
rootdir=os.getcwd()
rootdir+= "/datasets"
os.chdir(rootdir)

Autosaving every 180 seconds


In [2]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

In [3]:
ls *.gz

ground_truth.csv.gz  sf_output.txt.gz  sl_output.txt.gz


In [4]:
!zcat sf_output.txt.gz| head

USER|ANTENNA_ID_0|COUNT_0|ANTENNA_ID_1|COUNT_1|ANTENNA_ID_2|COUNT_2|ANTENNA_ID_3|COUNT_3|ANTENNA_ID_4|COUNT_4|ANTENNA_ID_5|COUNT_5|ANTENNA_ID_6|COUNT_6|ANTENNA_ID_7|COUNT_7|ANTENNA_ID_8|COUNT_8|ANTENNA_ID_9|COUNT_9|MOBILITY_DIAMETER|EXPOSED|EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03|3415|21|2969|5|1320|3|2471|3|1115|2|-1|65535|-1|65535|-1|65535|-1|65535|-1|65535|1063|False|False
FFFF51FDDD55A2E24D074DE30C4798E9|1207|15|1171|14|1820|9|583|8|2026|5|1066|4|1831|4|1625|2|401|1|825|1|520|True|False
FFFE1EDDA25369CBC467536FA6A787FD|1847|22|1753|14|1422|11|1028|10|765|9|3072|9|862|8|2463|7|1416|6|871|5|990|True|False
FFFD0A912A959CDFCFFAB2F93BFBA435|1407|18|1702|8|1613|6|1704|5|1394|3|1398|3|818|2|958|2|726|1|848|1|491|True|False
FFFCB235D6D917DDF350F5B11CCB0EE8|17|74|174|33|1123|4|1178|4|173|3|890|2|503|1|1039|1|1228|1|1327|1|1613|True|False
FFFC9EC4F662C44D3006BA69C9F0DE85|876|19|1737|6|1744|5|326|1|378|1|503|1|1118|1|2216|1|2477|1|2962|1|3004|True|True
FFFB2F18F4CCF253C06B581EA518A7A5|2543|

In [5]:
!stat sf_output.txt.gz | head

  File: `sf_output.txt.gz'
  Size: 181994072 	Blocks: 355464     IO Block: 4096   regular file
Device: 831h/2097d	Inode: 68164921    Links: 1
Access: (0644/-rw-r--r--)  Uid: ( 3001/    juan)   Gid: ( 3001/    juan)
Access: 2016-06-08 14:35:07.115044438 +0000
Modify: 2016-05-13 00:07:52.000000000 +0000
Change: 2016-06-08 14:35:07.115044438 +0000
 Birth: -


In [6]:
file_sf = "sf_output.txt.gz"
file_sl = "sl_output.txt.gz"
file_gt = "ground_truth.csv.gz"

sf_table = pd.read_csv(file_sf,sep = "|",
                   header =0,
                     index_col =0)

sl_table = pd.read_csv(file_sl,sep = "|",
                   header =0,
                     index_col =0)

gt_table = pd.read_csv(file_gt,sep = "|",
                   header =0,
                     index_col =0)

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_table.columns = [col+"_gt" for col in gt_table.columns]

In [7]:
sf_table.shape, sl_table.shape, gt_table.shape

((3667451, 23), (2449242, 15), (2000285, 3))

In [8]:
#checking table consistency
np.in1d(gt_table.index.values,sf_table.index.values).sum()

2000285

In [9]:
np.in1d(gt_table.index.values,sl_table.index.values).sum()

2000285

In [10]:
sf_table.sample(10)

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
USER,,,,,,,,,,,,,,,,,,,,,
01737C4634D44B15BDE3B4D1704B7A86,763,3,350,1,481,1,3444,1,-1,65535,...,65535,-1,65535,-1,65535,-1,65535,2015,False,False
F94E1DDF50F2FED57BB9F49C5082F612,31,10,2181,9,660,6,1743,5,2098,3,...,1,67,1,99,1,235,1,1603,True,True
ED73ACA101DF2A0722B321BAC792A361,3694,21,821,17,3454,17,3549,17,446,16,...,12,815,12,568,11,751,9,1820,True,False
D2EC7962039DC318369B17C105D020CA,642,18,1924,8,2011,7,1596,4,2025,3,...,2,1396,2,2091,2,2832,2,1334,True,True
ED1F4E5A278E80E1C70329928EAB4F82,271,19,2659,11,401,4,398,3,270,2,...,1,592,1,653,1,993,1,1148,True,False
594979EB2DBCB0BCD8815BD2D5180D7E,2237,26,1916,17,1908,14,1973,9,1978,7,...,3,394,2,419,1,490,1,1600,True,False
25C009AE4C33CB348C1B86ED0B494FE8,2182,31,148,24,1724,21,2443,19,2033,11,...,7,1674,4,2487,4,1905,3,1310,True,False
F13667C6DAA7D4E5D2FCA6C62CFCEA7E,3558,44,3166,16,1341,15,832,6,997,1,...,1,2018,1,-1,65535,-1,65535,794,True,True
25FCA8BFC511E0A9011EBFCD184EC3AB,3142,9,416,7,3112,6,3141,4,3210,4,...,3,3060,3,3102,3,1548,2,1945,True,False


In [11]:
sl_table.head()

,CallsWeekDaylight,CallsWeekNight,CallsWeekend,TimeWeekDaylight,TimeWeekNight,TimeWeekend,CallsWeekDaylight_EPI,CallsWeekNight_EPI,CallsWeekend_EPI,TimeWeekDaylight_EPI,TimeWeekNight_EPI,TimeWeekend_EPI,TOTAL_USERS,EPI_USERS,EXP_USERS
LineKeyOrigin,,,,,,,,,,,,,,,
000004F96010462FA7A71C7784E27476,0,0,1,0,0,34,0,0,0,0,0,0,1,0,1
000007EEE7895E17937CFF86379C8E17,2,0,0,144,0,0,2,0,0,144,0,0,1,1,1
00000837B868F926F67BD440A0845C1D,1,6,3,47,160,109,0,0,0,0,0,0,3,0,2
00000C5D3BFBEBD919F6F68386C3DD5E,4,0,0,281,0,0,0,0,0,0,0,0,1,0,0
0000129F4191FFD0963774D2307C7510,2,0,0,66,0,0,0,0,0,0,0,0,2,0,2


In [12]:
gt_table.head()

,ANTENNA_ID_gt,EPIDEMIC_gt,EXPOSED_gt
USER,,,
FFFF51FDDD55A2E24D074DE30C4798E9,470,0,0
FFFC3133FAEF588B9FA75487C864F774,1036,0,1
FFFB2F18F4CCF253C06B581EA518A7A5,497,1,1
FFFB2C0FEE677EE733DDD85F11C5A14C,214,0,1
FFFB27D77413E57C38D14D876C3DE8CF,2946,0,1


In [13]:
# para poder usar el estimador MultionmialNB, paso todos los valores negativos en las antenna_ids de sf_table a 0
for col in [col for col in sf_table.columns.values if "_ID" in col]:
    sf_table[col] = sf_table[col].replace(-1,0)

In [14]:
X_train = pd.concat([gt_table,sf_table, sl_table], axis=1, join='inner')
y_train = X_train[[col for col in X_train.columns if "_gt" in col]].copy()
X_train = X_train[[col for col in X_train.columns if not ("_gt" in col)]]

## Choosing different fitting objectives
We have enough information to attack the following questions:
* Predict if user A used to be exposed.
* Predict if user A used to be epidemic.
* Predict if user A used to have Antenna N as a base one. (antenna number has to be given and given that we have more than 4500 antenna, classes will be EXTREMELY unbalanced in this scenario )

In [15]:
y_target = gt_table['EPIDEMIC_gt'].values
#y_target = gt_table['EXPOSED_gt'].values
X_train['y'] = y_target

In [16]:
del sf_table, sl_table, gt_table

In [17]:
X_train.head()

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,CallsWeekDaylight_EPI,CallsWeekNight_EPI,CallsWeekend_EPI,TimeWeekDaylight_EPI,TimeWeekNight_EPI,TimeWeekend_EPI,TOTAL_USERS,EPI_USERS,EXP_USERS,y
000004F96010462FA7A71C7784E27476,2937,36,1787,22,3314,19,2919,3,3056,1,...,0,0,0,0,0,0,1,0,1,0
000007EEE7895E17937CFF86379C8E17,625,6,33,5,1915,5,624,3,1507,2,...,2,0,0,144,0,0,1,1,1,0
00000837B868F926F67BD440A0845C1D,650,102,1955,48,1915,26,2037,3,1649,2,...,0,0,0,0,0,0,3,0,2,1
00000C5D3BFBEBD919F6F68386C3DD5E,1325,3,2699,1,3227,1,3434,1,3583,1,...,0,0,0,0,0,0,1,0,0,0
0000129F4191FFD0963774D2307C7510,2226,18,306,12,891,11,169,10,364,10,...,0,0,0,0,0,0,2,0,2,0


## Correlation and var in input table
Se sabe que las columnas con  mucha correlacion son malas para los modelos lineales (no tanto asi con RF o XGB ). A estas columnas seria bueno cambiarlas, editarlas/modificarlas para no seguir usando esta info.

Las columnas de "poca" varianza no aportan informacion basicamente y por eso se dropean.

In [18]:
%%time
corr_series = X_train.corr().abs()
#lo paso a una serie
corr_series = corr_series.unstack()
#ordeno por los valores mas altos
corr_series = corr_series.sort_values(ascending = False) 
#es obvio que la diagonal tenia correlacion ==1, asi que elimino aquellos valores que eran de la diagonal
#tambien saco los valores que estan "arriba" de la diagonal, pero haciendo la comparacion entre strings
corr_series = corr_series[(corr_series.index.get_level_values(0) != corr_series.index.get_level_values(1))\
                         & (corr_series.index.get_level_values(0) <= corr_series.index.get_level_values(1))]


CPU times: user 21.5 s, sys: 1.25 s, total: 22.7 s
Wall time: 23.1 s


In [19]:
corr_series.head(20)

EXP_USERS              TOTAL_USERS             0.984376
COUNT_8                COUNT_9                 0.918492
COUNT_7                COUNT_8                 0.908408
COUNT_6                COUNT_7                 0.895939
COUNT_5                COUNT_6                 0.872578
CallsWeekDaylight      TimeWeekDaylight        0.857456
CallsWeekDaylight_EPI  TimeWeekDaylight_EPI    0.853580
EPI_USERS              EXP_USERS               0.851134
COUNT_7                COUNT_9                 0.834363
EPI_USERS              TOTAL_USERS             0.828276
CallsWeekNight         TimeWeekNight           0.825796
CallsWeekNight_EPI     TimeWeekNight_EPI       0.823400
COUNT_4                COUNT_5                 0.815779
COUNT_6                COUNT_8                 0.813874
CallsWeekend_EPI       TimeWeekend_EPI         0.794761
CallsWeekend           TimeWeekend             0.794346
COUNT_3                COUNT_4                 0.793586
COUNT_5                COUNT_7                 0

In [20]:
# como resultado de esto decidimos cambiar las columnas
# [ exp_USERS ] y repetir el algo
# la idea es tomar la diferencia entre las dos columnas correlacionados pues puede aportar nueva info.
X_train['EXP_USERS'] = X_train['TOTAL_USERS'] - X_train['EXP_USERS']

In [21]:
%%time
# miramos ahora las columnas con poca varianza ie. que aportan poca info

var_matrix = X_train.var()
#imprimimos aquellas que esten por abajo de cierto percentil
perc = 0.5
var_matrix[var_matrix<perc]

CPU times: user 41.5 s, sys: 6.67 s, total: 48.1 s
Wall time: 48.6 s


In [22]:
# como resultado de esto decidimos dropear las columnas, por poca varianza..
# [ EXPOSED ]
X_train.drop('EXPOSED',axis=1, inplace=True)

In [23]:
%%time
#con decomposicion qr tratamos de ver si existe alguna columna que sea comb linear de las demas
linear_test = np.linalg.qr(X_train.sample(1e6))[1]
#notar que devuelve la tabla de tamanyo N`columnas x Ncolumnas

#sumo a traves de las columnas para que me de el valor absoluto sumado de c/fila
linear_test = abs(linear_test.sum(axis=1))<1e-2
#si hubiese alguan que sea linearcomb entonces tendria que aparecer que toda la fila es de ceros
for i, col in enumerate(X_train.columns):
    if linear_test[i] == True:
        print(col)

/home/juan/miniconda2/lib/python2.7/site-packages/pandas/core/generic.py:2569: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


CPU times: user 19 s, sys: 7.15 s, total: 26.1 s
Wall time: 9.82 s


## First Simple Classification Tests
La idea es probar tres clasificadores basicos y tener una medida de precision, recall, F1-score p/c/ estimador:

* **Clasificador 1**: con la proporción de gente que vive en la zona endémica en el "presente", asignar la gente al azar respetando esa proporción i.e. $X \sim Bernoulli(\hat{p}) s.t. p$ is the mean of endemic users in the dataset

* **Clasificador 2**: usando la información del "presente", si la persona vive en la zona endémica en el presente, predecir que va a ocurrir lo mismo en el "pasado" i.e. un estimador que devuelve el feature de endemicidad actual para predecir la endemicidad en el pasado.


* **Clasificador 3**: usar 2 features del "presente", (i) si la persona vive en la zona endémica, (ii) cual es la proporción de sus contactos que viven en la zona endémica.

# Clf 1

In [29]:
from sklearn.metrics import *
from scipy import stats

/home/juan/mobility-study/scikit-learn/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 38] Function not implemented.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [43]:
p_hat = X_train['EPIDEMIC'].mean()
#the output vector, i.e. the estimated epidemic users
y_hat = stats.bernoulli.rvs(p_hat, size=len(X_train))

In [44]:
print("A priori probability of bein epidemic is %s" %p_hat)
print(classification_report(y_target, y_hat ))

A priori probability of bein epidemic is 0.293362195887
             precision    recall  f1-score   support

          0       0.71      0.71      0.71   1413371
          1       0.29      0.29      0.29    586914

avg / total       0.58      0.58      0.58   2000285



# Clf 2

In [45]:
y_hat = X_train['EPIDEMIC'].values
print(classification_report(y_target, y_hat ))

             precision    recall  f1-score   support

          0       0.71      0.71      0.71   1413371
          1       0.29      0.29      0.29    586914

avg / total       0.59      0.59      0.59   2000285



# Clf 3
adf

adsf

asdf

In [48]:
'number of users who are and have been epidemic is %s'% (X_train['EPIDEMIC'].values == y_target).sum(), "dataset size is %s" % len(X_train)

('number of users who are and have been epidemic is 1171065',
 'dataset size is 2000285')

## Cross, test, train splt

In [49]:
%%time
#parto efectivamente el dataset en 3 conjunots, uno para fitear todo. otro para entrenar+transform y el ultimo
#para transform +val
msk = np.random.rand(len(X_train)) < 0.3333
X_fit = X_train[msk]
X_train =  X_train[~msk]
msk2 = np.random.rand(len(X_train)) < 0.5
X_val =  X_train[msk2]
X_train =  X_train[~msk2]

CPU times: user 1.46 s, sys: 544 ms, total: 2.01 s
Wall time: 2.05 s


## Count Featurizer on Antennas
nos quedamos con las columnas de antennas y en graphlab aplicamos el algo de CountFeaturizer para cada categoria de
Antenna_ID_N


In [50]:
%%time
## Count Featurizer on Antennas
#nos quedamos con las columnas de antennas y pasamos todo a Graph Lab.
import graphlab as gl
from graphlab.toolkits.feature_engineering import *
ant_cols = [col for col in X_train.columns if "ANTENNA_ID" in col]
ant_sframe_fit = gl.SFrame(X_fit[ant_cols + ['y']])
ant_sframe_train = gl.SFrame(X_train[ant_cols + ['y']])
ant_sframe_val = gl.SFrame(X_val[ant_cols + ['y']])

A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-06-14 00:17:59,281 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1465863477.log


CPU times: user 7.21 s, sys: 1.51 s, total: 8.72 s
Wall time: 7.14 s


In [51]:
%%time
countfeat = gl.feature_engineering.create(ant_sframe_fit, 
               CountFeaturizer(target='y'))

# Transform the train set. This is the dataset I will train my classifier on
transformed_ant_train = countfeat.transform(ant_sframe_train)
transformed_ant_val = countfeat.transform(ant_sframe_val)

del ant_sframe_fit,ant_sframe_train,ant_sframe_val

#por alguna razon guarda los valores de probabilidad como una lista de un unico valor
for col in [col for col in transformed_ant_train.column_names() if "prob_" in col]:
    transformed_ant_train[col] = transformed_ant_train[col].apply(lambda x: x[0]) 
    transformed_ant_val[col] = transformed_ant_val[col].apply(lambda x: x[0]) 

#me quedo solo con los valores de probabilidad.
transformed_ant_train = transformed_ant_train[[col for col in transformed_ant_train.column_names() if "prob_" in col]]
transformed_ant_val = transformed_ant_val[[col for col in transformed_ant_train.column_names() if "prob_" in col]]

transformed_ant_val = transformed_ant_val.to_dataframe()
transformed_ant_train = transformed_ant_train.to_dataframe()

CPU times: user 58.6 s, sys: 8.91 s, total: 1min 7s
Wall time: 37.3 s


In [52]:
X_val[ant_cols] = transformed_ant_val.values
X_train[ant_cols] = transformed_ant_train.values
del transformed_ant_train, transformed_ant_val

## Scikit estimator fit and cross-validation

In [53]:
from sklearn.naive_bayes import *
#from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.neural_network import *
from sklearn.cross_validation import *
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import *
from sklearn.preprocessing import label_binarize, scale, StandardScaler
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.pipeline import Pipeline

/home/juan/mobility-study/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/juan/mobility-study/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Create new variables
These would be more "real" variables i.e. variables that would be "humanly" explainable

In [54]:
X_train.columns
#tenemos 38 cols

Index([u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9',
       u'MOBILITY_DIAMETER', u'EPIDEMIC', u'CallsWeekDaylight',
       u'CallsWeekNight', u'CallsWeekend', u'TimeWeekDaylight',
       u'TimeWeekNight', u'TimeWeekend', u'CallsWeekDaylight_EPI',
       u'CallsWeekNight_EPI', u'CallsWeekend_EPI', u'TimeWeekDaylight_EPI',
       u'TimeWeekNight_EPI', u'TimeWeekend_EPI', u'TOTAL_USERS', u'EPI_USERS',
       u'EXP_USERS', u'y'],
      dtype='object')

## Create new features
these would be "synthetic" variables

In [55]:
def process_X_files(tables, scaled=False, poly2=False,train_size = 0.8):
    #como input tiene que entrar la tabla sin splitea en train_test aun
    #tiene que suceder que haya consistencia entre las columnas de c/dataframe tambien
    
    
    calls_cols = [col for col in X_train.columns if "Calls" in col] 
    time_cols = [col for col in X_train.columns if "Time" in col]
    epi_calls_cols = [col for col in X_train.columns if "Calls" in col and "EPI" in col] 
    epi_time_cols = [col for col in X_train.columns if "Time" in col and "EPI" in col]
    antenna_cols = [col for col in X_train.columns if "ANTENNA_" in col]
    count_cols = [col for col in X_train.columns if "COUNT_" in col]
    condition_cols = [col for col in X_train.columns if "EXPOSED" == col or "EPIDEMIC" == col]
    
    categorical_cols = condition_cols
    numerical_cols = [col for col in table.columns if col not in categorical_cols]

    print("First dataframe is %s big" % str(train_table.shape))
    #print("Test dataframe is %s big" % str(test_table.shape))
    
    X_train_categorical = train_table[categorical_cols].values
    X_train_numeric = train_table[numeric_cols].values
    
    if scaled ==True :
            min_max_scaler = MinMaxScaler().fit(X_train_count_time)
            X_train_count_time = min_max_scaler.transform(X_train_numeric)
    
    #hacemos interacciones polinomiales sobre las columnas de count/time
    if poly2 == True:
        #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
        #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
        #como tope para aplicar interacciones polinomiales

        poly2_transform = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
        max_cols = 50

        if  (X_train_count_time.shape[1] <= max_cols):
            max_cols = X_train_count_time.shape[1]
        poly2_transform.fit(X_train_count_time[:,0:max_cols])

        X_train_poly2 = poly2_transform.transform(X_train_count_time[:,0:max_cols])
        X_train_count_time = np.concatenate( ( X_train_poly2, X_train_count_time[:,max_cols:]),axis=1) 

    X_train = np.concatenate( ( X_train_count_time, X_train_categorical),axis=1)
    
    processed_tables = [X_train]
    
    #aca basicamente replicamos lo anterior pero para todo el resto de las tablas y utilizando los fits que ya tenemos

    for i in range(len(tables)):
        # skip the X_train table which comes in the first postion and will behave as our special fitting table
        if i ==0:
            continue
        print(i)
        X_val_categorical = tables[i][categorical_cols].values
        X_val_count_time = tables[i][count_time_cols].values

        if scaled ==True :
            X_val_count_time = min_max_scaler.transform(X_val_count_time)

        #hacemos interacciones polinomiales sobre las columnas de count/time
        if poly2 == True:
            #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
            #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
            #como tope para aplicar interacciones polinomiales

            X_val_poly2 = poly2_transform.transform(X_val_count_time[:,0:max_cols])
            X_val_count_time = np.concatenate( ( X_val_poly2, X_val_count_time[:,max_cols:]),axis=1) 

        X_val = np.concatenate( ( X_val_count_time, X_val_categorical),axis=1)
        
        processed_tables = processed_tables + [X_val]
    
    for i in range(len(processed_tables)):    
        print("Table {0} shape is {1}".format( i,str(processed_tables[i].shape)))
#        print("Table %s categorical shape is %s" % str(X_val_categorical.shape))
#        print("Val non-categorical shape is %s" % str(X_val_count_time.shape))
#        print("Val categorical shape is %s" % str(X_val_categorical.shape))
    
    return processed_tables

## GradBoost model

In [56]:
## seteo los generalizadores de CV
##genero los estimadores
W = np.array([15 if i == 1 else 1 for i in y])
#model = GradientBoostingClassifier()

gradboost = GradientBoostingClassifier( loss = 'deviance',n_estimators=20, 
                                       max_depth = 20
                                 )
svc = LinearSVC(C=2.0, tol = 1e-4,
                class_weight = 'balanced'
               )

svc2 = SVC(C=2.0, kernel = 'rbf' ,decision_function_shape='ovr',tol = 1e-5,
                class_weight = 'balanced', max_iter = 1000
               )


NameError: name 'y' is not defined

# Multinomial Naive Bayes - Benchmark - Cross Validation
mnb es bastante rapido y eficiente para rapidamente resultados en problemas de clasificacion supervisada

In [79]:
start_time = time.time()
param_grid = {'alpha':[0,1.0, 1e-1,1e1], 'fit_prior': [True,False],
             }
estimator  = MultinomialNB( )

clf =GridSearchCV(estimator, param_grid, scoring='roc_auc', fit_params=None, n_jobs=10, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')
n_iter_search =60

#how many parameters to randomly search for
n_iter_search = 45

#clf = RandomizedSearchCV(estimator, param_distributions=param_grid,
#                                   n_iter=n_iter_search, n_jobs =-1, verbose=3)
X = X_train[X_train.columns[:-1]].values
Y = X_train['y'].values

#X = X_train_categorical
clf.fit(X,Y)


#random_search.fit(X_train,y_train)
elapsed_time =  time.time() - start_time

print('Random Search took %s seconds to run' % elapsed_time)


all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))


X_test = X_val[X_train.columns[:-1]].values
Y_test = X_val['y'].values

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba.index = X_val.index
predicted_proba['class'] = clf.predict(X_test)
predicted_proba['gt'] =  Y_test
predicted_proba.index.name = "USER"
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro

past_epidemic = X_val[X_val['y']==1].index.values

start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
    #subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1]].index.values
    
    print(classification_report(predicted_proba.loc[index_segments[-1],"gt"].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,"gt"].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
print("* Rates for distinct percentage groups \n")
for i in cut_percentages:
    subtable = mobility_dict[str(i)]

    print("{0} %% group rate: {1} size: {2}".format(str(i), 
                                np.in1d(subtable,past_epidemic).sum()*1.0/len(subtable),len(subtable))) 
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

    
    
all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.297282 -   2.9s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.294858 -   2.9s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.295935 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.297254 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.294880 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.295969 -   3.0s
[CV] alpha=0.5, fit_prior=True .......................................
[CV] ...........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   45.9s


[CV] .............. alpha=1.0, fit_prior=True, score=0.297282 -   2.9s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] .............. alpha=1.0, fit_prior=True, score=0.294858 -   2.9s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] .............. alpha=1.0, fit_prior=True, score=0.295935 -   2.9s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.297254 -   3.0s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.294880 -   2.9s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.295969 -   3.0s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] .............. alpha=0.1, fit_prior=True, score=0.297282 -   3.0s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] .

[Parallel(n_jobs=10)]: Done  36 out of  36 | elapsed:  2.2min finished


Random Search took 138.141034126 seconds to run
Grid Search took 0.000313997268677 seconds to run
             precision    recall  f1-score   support

          0       0.71      0.72      0.71     47103
          1       0.29      0.27      0.28     19520

avg / total       0.58      0.59      0.59     66623

             precision    recall  f1-score   support

          0       0.71      0.72      0.71     93945
          1       0.30      0.28      0.29     39301

avg / total       0.59      0.59      0.59    133246

             precision    recall  f1-score   support

          0       0.71      0.72      0.71    141314
          1       0.29      0.28      0.28     58555

avg / total       0.59      0.59      0.59    199869

             precision    recall  f1-score   support

          0       0.71      1.00      0.83    188401
          1       0.30      0.00      0.01     78091

avg / total       0.59      0.71      0.59    266492

             precision    recall  f1-score

In [78]:
coln = 1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print('\n std_deviation is')
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

alpha
0.00     0.500891
0.01     0.500891
0.10     0.500891
0.50     0.500891
1.00     0.500891
10.00    0.500911
Name: mean_score, dtype: float64

 std_deviation is
alpha
0.00     0.000066
0.01     0.000066
0.10     0.000066
0.50     0.000066
1.00     0.000066
10.00    0.000088
Name: mean_score, dtype: float64


# Results
* fitting prior has better mean and almost same std
* alpha should be only > 0. But difference between positive alphas is negligible, even on different exponential orders
* 

## Logistic Regression cross validation

In [54]:

clf =  LogisticRegressionCV(n_jobs =3, class_weight= 'balanced', scoring = 'f1',cv=3)

clf.fit(X_train,y_train)


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.67      0.85      0.75     43545
          1       0.40      0.19      0.25     22464

avg / total       0.58      0.63      0.58     66009

             precision    recall  f1-score   support

          0       0.64      0.75      0.69     83353
          1       0.40      0.28      0.33     48666

avg / total       0.55      0.58      0.56    132019

             precision    recall  f1-score   support

          0       0.63      0.67      0.65    124185
          1       0.39      0.35      0.37     73843

avg / total       0.54      0.55      0.55    198028

             precision    recall  f1-score   support

          0       0.62      0.56      0.59    164393
          1       0.38      0.44      0.41     99645

avg / total       0.53      0.52      0.52    264038

             precision    recall  f1-score   support

          0       0.64      0.66      0.65    415477
          1       0.39      0.36 

AttributeError: 'LogisticRegressionCV' object has no attribute 'grid_scores_'

## Cross Validate SGD

In [ ]:
param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]

In [37]:
start_time = time.time()

param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]
sgd = SGDClassifier(loss='modified_huber', penalty='elasticnet', 
             fit_intercept=True,  shuffle=True, 
                    n_jobs=3,learning_rate='optimal', power_t =2, eta0 =5,
                    class_weight='balanced', average=40)

clf =GridSearchCV(sgd, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)



predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


...

[CV]  n_iter=2, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.122741 -   3.5s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.132433 -   3.7s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135111 -   3.4s
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135908 -   3.1s


[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:  1.7min finished


             precision    recall  f1-score   support

          0       1.00      0.31      0.47    355992
          1       0.06      0.99      0.11     15152

avg / total       0.96      0.33      0.45    371144

This cell took 108.56432461738586 seconds to run


In [ ]:
threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))

In [38]:
clf.best_estimator_

SGDClassifier(alpha=0.005, average=40, class_weight='balanced', epsilon=0.1,
       eta0=5, fit_intercept=True, l1_ratio=0.0006000000000000001,
       learning_rate='optimal', loss='modified_huber', n_iter=3, n_jobs=3,
       penalty='elasticnet', power_t=2, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [39]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_iter
2.0    0.125741
3.0    0.132943
Name: mean_score, dtype: float64
n_iter
2.0    0.002609
3.0    0.002703
Name: mean_score, dtype: float64


### Results
* l1_ratio = cuanto mas chico mejor con lo cual la perdida l2 parece ser mejor
* alpha = 1e-3 es suficiente pues casi no afecta el score
* power_t = muy variado, no parece haber correlacion entre el tamanyo y el avg, mean_score
* eta0 = no afecta mucho pero parece ser que con ser >1 ya esta
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

In [51]:
len(predicted_proba)

371161

## Cross Validate NN

In [35]:
start_time = time.time()

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

param_grid = { 'alpha': [1e-1,5*1e-2,1e-2],
              'hidden_layer_sizes':[(50,),(15,5)],
              'learning_rate': ['adaptive',"invscaling"],
              "algorithm": ['adam'],'momentum': [1e-2, 1e-1, 0.5],
  'power_t': [1e-3, 5*1e-4, 1e-5], 'activation':['logistic','relu']
 }

mlp = MLPClassifier(shuffle=True, 
                 verbose=True)

clf =GridSearchCV(mlp, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 
Iteration 1, loss = 0.20405210
Iteration 1, loss = 0.20380843
Iteration 1, loss = 0.20378706
Iteration 1, loss = 0.20405210


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.5s
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.7s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   2.1s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.9s


KeyboardInterrupt: 

In [1]:
clf.best_estimator_

NameError: name 'clf' is not defined

In [52]:
# compare best parameters to tune
coln=1
cv_result.groupby(cv_result.columns[coln])['mean_score'].mean()
cv_result.groupby(cv_result.columns[coln])['mean_score'].std()

alpha
0.00     0.000007
0.01     0.000007
0.10     0.000007
0.50     0.000007
1.00     0.000007
10.00    0.000007
Name: mean_score, dtype: float64

### Results
* activation = logistic es 10% mejor
* alpha = 1e-2 el mejor 
* power_t = cuanto mas chico mejor, 1e-3 por lo menos
* hidden_layer_size = menos layers es mejor..?
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

## Bernoulli RBM features selection & Logit crossval

In [127]:
start_time = time.time()

## obs. este metodo es especial y asume que todos los valores son True/False o que \in [0,1]
# luego tengo que pensar en como tomar los features nuevamente.
# tampoco 'fittea' en la forma tradicional. Sino que se le puede tomar al y como un feature mas y esta red
# va 'modificando' todos los valores del X (minimizando la entropia) para dar un output. Luego corriendo 
# clf.gibbs(X_test) con el y_test como feature tmb, nos transforma la data para ver el output como la 'prediccion'
# obviamente no tiene probabilidades


#X = X_train[X_cols].values
#y = X_train['ground_truth'].values

df = X_train.drop(X_train[X_train[X_train.columns[0:3]].\
                                   sum(axis=1)==0].index)
df = df[X_cols + ['ground_truth']]

for col in X_train.columns[0:3]:
    df[col] = df[col]*1.0/df[df.columns[0:3]].sum(axis=1)

df[df.columns[3]] =  df[df.columns[3]]/df[df.columns[3]].max()
df[df.columns[4]] =  df[df.columns[4]]/df[df.columns[4]].max()

X = df[df.columns[:-1]].values
y = df['ground_truth'].values


param_grid = {'rbm__n_components': [256, 128,46,10],
   'rbm__n_iter':[15,10,5], 'rbm__learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1],
  'rbm__batch_size': [10e4,3*10e3, 1e3, 300],"logistic__C": [1.0, 10.0, 100.0] 
 }

rbm = BernoulliRBM(verbose=True)
logistic = LogisticRegression()
classifier = Pipeline([("rbm", rbm), ("logistic", logistic)])

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

clf =GridSearchCV(classifier, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

#este metodo NO tiene predicted proba, lo que hacemos es recorrer 
#predicted_labels = rbm.gibbs(X_test)[:,-1]
#real_labels = X_test[:,-1]
#print(classification_report(real_labels,predicted_labels ))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

finished = True

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
    
print('This cell took %s seconds to run' % all_time)


KeyboardInterrupt: 

In [ ]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

## ADaboost CV

In [41]:
rf_param_grid = {'n_estimators': [10, 30, 100, 300, 1000]}
boost_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                      'max_depth': [2, 3, 4, 5],
                      'min_samples_leaf': [1, 2, 3]}
ada_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                   'learning_rate': [0.1, 0.3, 1.0, 3.0]}

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#rf_est = RandomForestClassifier()
#rf_gs_cv = GridSearchCV(rf_est, rf_param_grid).fit(X_train, Y_train)
#print(rf_gs_cv.best_score_, rf_gs_cv.best_params_)
print('\n')
 
#boost_est = GradientBoostingClassifier()
#boost_gs_cv = GridSearchCV(boost_est, boost_param_grid).fit(X_train, y_train)
#print(boost_gs_cv.best_score_, boost_gs_cv.best_params_)
print('\n')
 
ada_est = AdaBoostClassifier()
ada_gs_cv = GridSearchCV(ada_est, ada_param_grid).fit(X_train, y_train)
print(ada_gs_cv.best_score_, ada_gs_cv.best_params_)
print('\n')

KeyboardInterrupt: 

In [42]:
start_time = time.time()


param_grid = {'n_estimators': [200,600,60,45,30,15],
 'learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1]
 }

aboost = AdaBoostClassifier(algorithm='SAMME.R')

clf =GridSearchCV(aboost, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

print('This cell took %s seconds to run' % all_time)

finished = True
#este flag es basicamente para chequear 

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

finished = True

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................


KeyboardInterrupt: 

In [23]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


criterion
gini    0.802919
Name: mean_score, dtype: float64
criterion
gini    0.000598
Name: mean_score, dtype: float64


## CrossV SVC

In [ ]:
start_time = time.time()

param_grid = [{'kernel': ['rbf'], 'gamma': list(10.0 ** np.arange(-2, 3)),
                     'C': list(10.0 ** np.arange(0, 4))},
                    {'kernel': ['poly'], 'C': list(10.0 ** np.arange(0, 4)), 'degree'[2,3,4]}]

svc = SVC(shuffle=True, probability=True,decision_function_shape = 'ovr',
           verbose=True, class_weight='balanced'
          )

clf =GridSearchCV(svc, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

In [ ]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


In [214]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

#model = model.fit(X,y,sample_weight=W)

W = np.array([10 if i == 1 or i ==2  else 1 for i in y_mini])
gradboost.fit(X_mini,y_mini, sample_weight=W)

elapsed_time =   time.time() - start_time
print('GradientBoosting took %s seconds to run' % elapsed_time)

#validated =  cross_val_score(gradboost,X,y,cv=5, scoring = "f1_weighted")


all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

GradientBoosting took 117.26272392272949 seconds to run
This cell took 117.26284193992615 seconds to run


## Timings
* 5s con  10 n_estimadores, 15 max_depth y X.sample(0.2)
* 15s con 10 n_estimadores, 15 max_depth y X.sample(0.2)
* 47s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 35s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 117 con 20 n_estimadores, 20 max_depth y X.sample(0.4)

In [31]:
X_train['ground_truth'].value_counts()

0    128153
1      7064
Name: ground_truth, dtype: int64

## Cross Validate - Random Forest

In [ ]:
param_grid = {'criterion': ['gini'], 'n_estimators': [15,30,50],
  'max_features': ["auto", "log2"], "bootstrap": [ False, True],
    "min_samples_leaf": np.append(np.random.randint(3,15,3),[3]),'max_depth':[10,20,30], 
               "class_weight": ['balanced']
              }

In [86]:
start_time = time.time()

param_grid = {'criterion': ['gini'], 'n_estimators': [25],
  'max_features': ["auto"], "bootstrap": [ False],
    "min_samples_leaf":[6,12],'max_depth':[15,25], 
               "class_weight": ['balanced']
              }
rforest  = RandomForestClassifier( )

clf =GridSearchCV(rforest, param_grid, scoring='roc_auc', fit_params=None, n_jobs=8, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf2 = RandomForestClassifier()

X = X_train[X_train.columns[:-1]].values
Y = X_train['y'].values

#X = X_train_categorical
clf.fit(X,Y)


#random_search.fit(X_train,y_train)
elapsed_time =  time.time() - start_time

print('Random Search took %s seconds to run' % elapsed_time)


all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))


X_test = X_val[X_train.columns[:-1]].values
Y_test = X_val['y'].values

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba.index = X_val.index
predicted_proba['class'] = clf.predict(X_test)
predicted_proba['gt'] =  Y_test
predicted_proba.index.name = "USER"
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro

past_epidemic = X_val[X_val['y']==1].index.values

start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
    #subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1]].index.values
    
    print(classification_report(predicted_proba.loc[index_segments[-1],"gt"].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,"gt"].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
print("* Rates for distinct percentage groups \n")
for i in cut_percentages:
    subtable = mobility_dict[str(i)]

    print("{0} %% group rate: {1} size: {2}".format(str(i), 
                                np.in1d(subtable,past_epidemic).sum()*1.0/len(subtable),len(subtable))) 
    

clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

    
    
all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV] bootstrap=False

[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  4.2min remaining:  -19.6s


[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV]  bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced, score=0.500056 - 4.2min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  4.3min remaining:  -19.9s


[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced, score=0.498584 - 4.2min


...

[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced, score=0.500189 - 5.3min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  9.7min remaining:  -44.7s


[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced, score=0.499681 - 5.3min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  9.8min remaining:  -45.1s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:  9.8min finished


Random Search took 1023.46121788 seconds to run
Grid Search took 0.000302076339722 seconds to run
             precision    recall  f1-score   support

          0       0.71      0.92      0.80     47125
          1       0.29      0.08      0.13     19498

avg / total       0.58      0.67      0.60     66623

             precision    recall  f1-score   support

          0       0.71      0.77      0.74     94223
          1       0.29      0.23      0.26     39023

avg / total       0.59      0.61      0.60    133246

             precision    recall  f1-score   support

          0       0.71      0.66      0.68    141157
          1       0.30      0.35      0.32     58712

avg / total       0.59      0.57      0.57    199869

             precision    recall  f1-score   support

          0       0.71      0.57      0.63    188258
          1       0.29      0.43      0.35     78234

avg / total       0.59      0.53      0.55    266492

             precision    recall  f1-score

In [85]:
np.bincount(Y)[0]*1.0/np.bincount(Y).sum(), np.bincount(Y)[1]*1.0/np.bincount(Y).sum()

(0.70631558433771058, 0.29368441566228937)

In [90]:
clf2 = RandomForestClassifier()
clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)

RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=25, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=12, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=6,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [92]:
#OJO esta parte con poly = True no funciona.
feature_importance = pd.DataFrame(clf2.feature_importances_,columns=['prob'])
feature_importance = feature_importance.sort_values(by = 'prob',ascending=False)
selected_features = X_train.columns[feature_importance.index]

#guardo los labels como estan en las tablas (con numeros y no palabras)
feature_importance['labels'] = selected_features.values

In [93]:
feature_importance

,prob,labels
2,0.047771,ANTENNA_ID_1
0,0.047296,ANTENNA_ID_0
6,0.046120,ANTENNA_ID_3
4,0.045383,ANTENNA_ID_2
20,0.045289,MOBILITY_DIAMETER
8,0.043635,ANTENNA_ID_4
10,0.040790,ANTENNA_ID_5
25,0.040621,TimeWeekDaylight
12,0.040533,ANTENNA_ID_6
14,0.038278,ANTENNA_ID_7


In [43]:
X_train.shape,y_train.shape

((1600644, 38), (1600644,))

In [35]:
X_train.shape

(1600644, 38)

In [51]:
clf.best_estimator_, clf.best_score_

(RandomForestClassifier(bootstrap=False, class_weight='balanced',
             criterion='gini', max_depth=15, max_features='auto',
             max_leaf_nodes=None, min_samples_leaf=4, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 0.54557839409818476)

In [17]:
a = classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values )

In [60]:
coln = 3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_estimators
20.0    0.537925
35.0    0.541880
Name: mean_score, dtype: float64
n_estimators
20.0    0.00539
35.0    0.00419
Name: mean_score, dtype: float64


### Results
* NO escalar (normalizar, restar la media  dividir por la std, etc) los resultados pues REVIENTAN los scores.
* bootstrap  = False es 5% mejor
* min_samples_leaf = mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators = aumentar mas de 30 no tendria mucho sentido
* citerion = entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features = no afecta al score. con auto esta bien
* max_depth =  mas es mejor. intentaria probar con >15

## Cross Validate ExtraTreeClassifier

In [94]:
predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_train)[:,1]
predicted_proba['class'] = clf.predict(X_train)
predicted_proba['ref_hash'] =  train_table.index.values
predicted_proba['y'] =  y_train
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro
past_epidemic = test_table[test_table['y']>0]['ref_hash'].values


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],'y'].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
print("Rates for %s app %s " % (appid_name[application_id], application_id))   
for i in cut_percentages:
    subtable = mobility_dict[str(i)]
    
    print("{0} %% group rate: {1} size: {2}".format(str(i), subtable.isin(past_epidemic).sum()*1.0/len(subtable), 
                                       len(subtable))) 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/metrics/classification.py:1117: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.00      0.00      0.00     26720
          1       0.25      1.00      0.40      8738

avg / total       0.06      0.25      0.10     35458

             precision    recall  f1-score   support

          0       0.97      0.43      0.60     66920
          1       0.07      0.76      0.14      3996

avg / total       0.92      0.45      0.57     70916

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    105255
          1       0.00      0.00      0.00      1119

avg / total       0.98      0.99      0.98    106374

             precision    recall  f1-score   support

          0       0.98      1.00      0.99    139582
          1       0.00      0.00      0.00      2250

avg / total       0.97      0.98      0.98    141832

Rates for dafiti app 189 
10 %% group rate: 0.10429240227875233 size: 35458
20 %% group rate: 0.024479666083817474 size: 70916
30 %% group r

In [100]:
coln = 2
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

alpha
0.00     0.000000
0.01     0.242260
0.10     0.242266
0.50     0.242220
1.00     0.242200
10.00    0.241269
Name: mean_score, dtype: float64
alpha
0.00     0.000000
0.01     0.017568
0.10     0.017598
0.50     0.017578
1.00     0.017570
10.00    0.017780
Name: mean_score, dtype: float64


## Cross validate MNB

In [72]:
start_time = time.time()

# pero tenemos que transformar a y en un vector canonico indicando en 1 en la posicion correspondiente a su categoria
#y =  label_binarize(X_train['ground_truth'].values.astype(int),
#                    classes=list(set(X_train['ground_truth'].values.astype(int))))

perc = 0.3
mini_ind = np.random.choice(X_train.shape[0],int(perc*X_train.shape[0]),replace=False)
#X_mini =  X[mini_ind,:]
#y_mini = y[mini_ind]


clf = MultinomialNB(
        )

clf2 = MultinomialNB()

validated = []
# Only take the first fold.
n_folds = 4
for i in range(n_folds):
    # Break up the dataset into non-overlapping training (75%) and testing
    # (25%) sets.
    skf = StratifiedKFold(y_train, n_folds=n_folds)
    train_index, test_index = next(iter(skf))

    X_traincv = X_train[train_index]
    y_traincv = y_train[train_index]

    n_classes = len(np.unique(y_traincv))

    clf.fit(X_traincv,y_traincv)
    predictions = clf.predict(X_traincv)
        
    expected = y_traincv

    print(classification_report(expected, predictions))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['y'] = y_test
predicted_proba['class'] = clf.predict(X_test)

clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)


#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

This cell took 1.06442093849 seconds to run


In [76]:
threshold = 0.99
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))


             precision    recall  f1-score   support

          0       0.99      0.87      0.93    185744
          1       0.15      0.73      0.26      5906

avg / total       0.96      0.87      0.91    191650



In [60]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

svc2.fit(X_mini,y_mini)
elapsed_time =   time.time() - start_time
print('SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

SVC took 362.8217749595642 seconds to run
This cell took 362.8219118118286 seconds to run


## Timings
* 40k samples : 360s
* 

In [75]:
start_time =  time.time()
lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

Logit took 1073.0756621360779 seconds to run


In [ ]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

svc.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Linear SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

Logit took 89.15975689888 seconds to run
Linear SVC took 804.7178750038147 seconds to run

In [ ]:
y.value_counts

# Todo
* evaluate hit_rate and 
* tune adaboost, bernoulliRBM
* xgboost
* libffm
* SVC muy lento.. speed up in AWS
